# TP d'optimisation sous incertitude - ENAC - M2RO - 2021


## Avant de commencer...


L'implémentation sera faite en utilisant le langage de programmation <tt>python</tt> dans ce <tt>notebook</tt> et la bibliothèque <tt>pulp</tt> (https://pypi.org/project/PuLP/). Pour chaque question, un  squelette vous est fourni et doit être complété. 

<b>Il est attendu un compte rendu avec les réponses à toutes les questions (5 pages max) et la remise des codes via le notebook.</b>


La cellule ci-dessous charge les librairies et effectue un test du bon fonctionnement de puLp. Si tout se passe bien, vous devez obtenir la sortie suivante:

<tt> === Programme de test (ENAC) ===<br>
Résolution d'un PLNE basique... Ok: x1 = 0.0, x2 = 2.0.<br>
=== Fin du programme === </tt>

In [1]:
# -*- encoding: utf-8 -*-

from __future__ import print_function

from airland import read_instance, read_scenarios
import pulp
from pulp import LpStatus

import argparse
import os
import sys
import verifier

import test_pulp
test_pulp.test_pulp()

=== Programme de test (ENAC) ===
Résolution d'un PLNE basique... Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mtds/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/ecb783947e664dbe89e4ba85d178fbc4-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/ecb783947e664dbe89e4ba85d178fbc4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 29 RHS
At line 37 BOUNDS
At line 38 ENDATA
Problem MODEL has 7 rows, 2 columns and 14 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-7) rows, 0 (-2) columns and 0 (-14) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value -2
After Postsolve, objective -2, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -2 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU sec

/home/mtds/anaconda3/lib/python3.11/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Le processus d'atterrissage d'un avion dans un aéroport

La figure ci-dessous illustre les principes de contrôle de la navigation d'un avion commercial sur un exemple de vol Orly-Toulouse. Pendant le vol de croisière le contrôle est assuré par un centre de contrôle régional, par exemple Paris et Bordeaux sur les deux avant derniers tronçons. Ensuite la tour de contrôle de l'aéroport prend le relai pour le contrôle d'approche.

Un avion $i$ entrant dans la zone de contrôle de l'aéroport à l'instant $a_i$ a une date d'atterrissage au plus tôt $e_i$ correspondant à la vitesse maximale qu'il peut atteindre et une date d'atterrissage au plus tard $d_i$ correspondant à la date d'atterrissage maximale possible compte-tenu du carburant disponible et de la possibilité de tourner en rond dans la zone d'attente. Par ailleurs, compte-tenu de sa vitesse de croisière, un avion a également une date d'atterrissage préférentielle $t_i$. En cas de violation de cette date préférentielle, des coûts unitaires en cas de retard  $g_i$ et en cas d'avance $h_i$ sont à payer. De plus, entre l'atterrissage de deux avions $i$ et $j$, un temps de séparation minimum noté $s_{ij}$ dépendant du type des deux appareils doit être respecté.

Le rôle des contrôleurs aériens est d'attribuer à tout avion une date d'atterrissage effective $T_i\in[e_i,d_i]$ ainsi qu'une piste d'atterrissage (lorsque plusieurs sont disponibles). Il n'est pas toujours possible de choisir $T_i=t_i$ pour tout avion en raison de possibilités d'engorgement suite à des arrivées proches de différents avions.

![title](images/procedure-atterrissage.png)
(Source: Cinq sur cinq numéro 15 - septembre 2016)


## 1. Le problème statique déterministe

Le problème statique détermine consiste, à partir d'un ensemble d'avion donné $P$, dont toutes les caractéristiques sont supposées connues à l'avance, et d'un ensemble de piste d'atterrissages $R$, à trouver les dates d'atterrissage qui respectent les contraintes décrites dans la section précédente et qui minimisent la somme pondérée des coûts d'avance et de retard. Il s'agit d'un problème d'ordonnancement NP-difficile, étudié dans l'article de Beasley *et al.* [1].  Un ensemble d'instances du problème proposé par les auteurs de cet article est téléchargeable sur le site de la OR-library (ces instances sont égalements fournies dans l'archive du TP), une bibliothèque de problèmes classiques de Recherche Opérationnelle (http://people.brunel.ac.uk/%7Emastjjb/jeb/orlib/airlandinfo.html).

<i> Exemple : Dans le cours 1, les données de l'instance "airland1.txt" à 10 avions et un exemple de solution  de coût total 1210 pour une seule piste d'atterrissage sont décrits.</i>

Dans cet article, le problème est formulé comme un programme linéaire en nombre entiers (PLNE). Une version simplifiée <b> et incomplète </b> est proposée ci-dessous.

En plus des paramètres déjà présentés, on introduit les variables de décisions binaires $x_{ij}$ qui indiquent que l'avion $i\in P$ atterrit avant l'avion $j\in P$ sur la même piste d'atterrissage et les variables binaires $y_{ir}$ qui indiquent que l'avion $i\in P$ atterrit sur la piste $r\in R$. On définit également des variables de décision continues : $T_i$ pour la date d'atterrissage de l'avion $i\in P$, $L_i$ pour le retard de l'avion $i\in P$, $E_i$ pour l'avance de l'avion $i\in P$ et $z$ pour l'objectif.

\begin{align}
  \min \quad & z\label{obj} & &\quad  (1) \\ % \sum_{i\in P} ( g_i L_i + h_u E_i ) \\
  \mbox{s.c. } \quad & T_i \geq e_i & \forall i\in P &\quad (2)\\
             &T_i \leq d_i & \forall i\in P&\quad  (3) \\
             % &x_{ij}+x_{ji}  \leq 3 - y_{ir} - y_{jr} & \forall i,j\in P,i<j,\forall r\in R&\quad(4)\\
             % &x_{ij}+x_{ji}  \geq y_{ir} + y_{jr} - 1 & \forall i,j\in P,i<j,\forall r\in R&\quad(5) \\
             % & \sum_{r\in R} y_{ir} = 1 & \forall i\in P&\quad(6)\\
             &T_j \geq T_i + s_{ij} x_{ij} - M_{ij} (1 - x_{ij} ) & \forall i,j\in P,i\neq j &\quad(4)\\
             & L_i \geq T_i-t_i & \forall i\in P&\quad(5)\\
             & E_i \geq t_i - T_i &\forall i \in P &\quad(6)\\
             & x_{ij}\in\{0,1\} & \forall i,j\in P,i\neq j &\quad(7)\\
             & y_{ir}\in\{0,1\} & \forall i \in P, \forall r\in R &\quad(8)\\
             & L_i, E_i, T_i \geq 0 &  \forall i \in P &\quad(9)
\end{align}

Les contraintes (2),(3) indiquent qu'un avion ne peut atterrir avant sa date d'arrivée au plus tôt ou après sa date d'arrivée au plus tard. Les contraintes (5),(6)  définissent les retards et les avances. Les domaines de définition des variables de décisions sont donnés par (7),(8),(9).

[1]: J.E. Beasley, M. Krishnamoorthy, Y.M. Sharaiha and
D. Abramson. Scheduling aircraft landings - the static case, Transportation Science, vol.34, 2000, pp180-197 (https://core.ac.uk/download/pdf/337240.pdf).


Nous commençons par lire une instance du problème. Pour tester avec d'autres instances il suffit de mentionner un autre nom de fichier contenu dans le répertoire <tt>instance</tt> de l'archive fournie.

In [289]:
import io
instance_name = 'instances/airland3.txt'
instance = open(instance_name)

runways=3

P, F, A, E, L, T, S, G, H = read_instance(instance)
print('Nombre d\'avions P='+str(P))
print()
print('Nombre de pistes R='+str(P))
print()
print('Largeur de la fenêtre de fixation des dates d\'atterrissage F='+str(F))
print()
print('Dates d\'apparition des avions A='+str(A))
print()
print('Dates d\'atterrissage au plus tôt E='+str(E))
print()
print('Dates d\'atterrissage au plus tard L='+str(L))
print()
print('Dates d\'atterrissage préférentielles T='+str(T))
print()
print('Pénalités unitaires de retard G='+str(G))
print()
print('Pénalités unitaires d\'avance H='+str(H))
print()

P = range(P)
print('matrice des temps de séparation')
import numpy as np
print(np.array(S))
#for i in P:
#    print(S[i])

R = range(runways)

Nombre d'avions P=20

Nombre de pistes R=20

Largeur de la fenêtre de fixation des dates d'atterrissage F=10

Dates d'apparition des avions A=[0, 82, 59, 28, 126, 20, 110, 23, 42, 42, 57, 39, 186, 175, 139, 235, 194, 162, 69, 76]

Dates d'atterrissage au plus tôt E=[75, 157, 134, 103, 201, 95, 185, 98, 117, 117, 132, 114, 261, 250, 214, 310, 269, 237, 144, 151]

Dates d'atterrissage au plus tard L=[486, 628, 561, 565, 735, 524, 664, 523, 578, 569, 615, 551, 834, 790, 688, 967, 818, 726, 607, 624]

Dates d'atterrissage préférentielles T=[82, 197, 160, 117, 261, 106, 229, 108, 132, 130, 149, 126, 336, 316, 258, 409, 338, 287, 160, 169]

Pénalités unitaires de retard G=[30.0, 10.0, 10.0, 30.0, 10.0, 30.0, 10.0, 30.0, 30.0, 30.0, 30.0, 30.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 30.0, 30.0]

Pénalités unitaires d'avance H=[30.0, 10.0, 10.0, 30.0, 10.0, 30.0, 10.0, 30.0, 30.0, 30.0, 30.0, 30.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 30.0, 30.0]

matrice des temps de séparation
[[99999    15    1


### Question 1.1 

L'objectif du programme linéaire ci-dessus est de minimiser la somme pondérée des coûts liés aux avances et retard.

<i>Ajouter une contrainte pour préciser l'expression de l'objectif $z$.</i>


### Question 1.2 

Le PLNE présenté ci-dessus est incomplet. En effet, il est possible que deux avions $i$ et $j$ ne soit pas séquencés ($x_{ij} = 0$ et $x_{ji} = 0$), ou bien qu'un avion ne soit affecté à aucune ou à plusieurs pistes ($y_{ir_1} = y_{ir_2} = 0$ ou $y_{ir_1} = y_{ir_2} = 1$).

<i> Ajouter <b>2</b> contraintes afin de régler ces deux problèmes.
    

### Question 1.3
    
<i>Expliquer la contrainte (4) en considérant que les $M_{ij}$ ont des valeurs très grandes. Quelle(s) bornes(s) peuvent être choisies pour les $M_{ij}$ afin d'obtenir un PLNE efficace et valide ?</i>
    


### Question 1.4 

<i> Programmer le PLNE ci-dessus avec la bibliothèque de modélisation PuLP en complétant la fonctoin <tt>get_problem_q1</tt> ci-dessous et tester le programme surles instances fournies. Donner un tableau avec pour chaque instance, la valeur objectif obtenue pour différents temps de calcul et différents nombres de pistes d'aterrissage. Qu'en concluez-vous sur l'impact de la taille de l'instance sur la résolution ?</i>


In [290]:
from pulp import lpDot, lpSum,\
     LpProblem, LpMinimize, \
     LpVariable, LpBinary, LpInteger

def get_problem_q1(**kargs):

    P = kargs['P']  # Liste d'avions (0, 1, 2, ..., N)
    R = kargs['R']  # Pistes d'atterrissage (0, 1, 2, ..., M)
    e = kargs['e']  # Dates d'atterrissage au plus tôt
    d = kargs['d']  # Dates d'atterrissage au plus tard
    t = kargs['t']  # Dates d'atterrissage préférentielles
    s = kargs['s']  # Délais de séparation
    g = kargs['g']  # Coûts unitaires de retard
    h = kargs['h']  # Coûts unitaires d'avance

    # -- Calcul des bigs M à faire !
    M = [[d[i]-e[j] for j in P] for i in P]

    # -- Création du problème
    problem = LpProblem("Ordonnancement des atterrissages - Le problème statique", LpMinimize)

    # -- Création des variables

    # T - Date d'atterrissage
    T = [LpVariable('T_{}'.format(i), e[i], d[i]) for i in P]

    # E - Avance de l'avion
    E = [LpVariable('E_{}'.format(i), 0) for i in P]
    # L - Retard de l'avion
    L = [LpVariable('L_{}'.format(i), 0) for i in P]

    # x - x_ij = 1 si l'avion i attérit avant l'avion j
    x = [[LpVariable('x_{}_{}'.format(i, j), cat=LpBinary) for j in P] for i in P]

    # y  - y_ir = 1 si l'avion i atterit sur la piste r
    y = [[LpVariable('y_{}_{}'.format(i, r), cat=LpBinary) for r in R] for i in P]

    # -- Création de l'objectif à faire !

    problem += lpDot(g, L) + lpDot(h, E)

    # -- Création des contraintes

    # Contraintes (2) et (3)
    # -> Inutiles car bornes données dans la création de T

    # Contraintes (4)
    for i in P:
        for j in filter(lambda x: x != i, P):
            problem += T[j] >= T[i] + s[i][j] * x[i][j] - M[i][j] * (1 - x[i][j])

    # Contraintes (5) et (6)
    for i in P:
        problem += L[i] >= T[i] - t[i]
        problem += E[i] >= t[i] - T[i]

    # Contraintes (7) - (9)
    # -> Inutiles car bornes données dans la création des variables
    
    # Contraintes (10) à ajouter !
    for i in P:
        for j in filter(lambda x: x != i, P):
            for r in R:
                problem += x[i][j] + x[j][i] >= y[i][r] + y[j][r] - 1

    # Contraintes d'affectation des avions aux pistes
    for i in P:
        problem += lpSum(y[i][r] for r in R) == 1

    
    print(problem)
    return problem

Construction des arguments pour la fonction get_problem et exécution de la méthode <tt>solve</tt> pour le solveur <tt>CBC</tt> (<i>Coin-or branch and cut</i>) avec <tt>msg=True</tt> activant l'affichage des sorties du solveur (vous pouvez voir le nombre de variables et de contraintes puis les inforemations sur les étapes de résolution). Un temps de calcul maximum peut être défini (ici 60 secondes). Faites varier ce temps pour voir la différence dans les solutions trouvées sur les différentes instances.

In [291]:
kargs = {'P': P, 'R': R, 'e': E, 'd': L, 't': T, 's': S, 'g': G, 'h': H}
problem = get_problem_q1(**kargs)
status = problem.solve(pulp.PULP_CBC_CMD(msg=False,maxSeconds=200))

Ordonnancement_des_atterrissages_-_Le_problème_statique:
MINIMIZE
30.0*E_0 + 10.0*E_1 + 30.0*E_10 + 30.0*E_11 + 10.0*E_12 + 10.0*E_13 + 10.0*E_14 + 10.0*E_15 + 10.0*E_16 + 10.0*E_17 + 30.0*E_18 + 30.0*E_19 + 10.0*E_2 + 30.0*E_3 + 10.0*E_4 + 30.0*E_5 + 10.0*E_6 + 30.0*E_7 + 30.0*E_8 + 30.0*E_9 + 30.0*L_0 + 10.0*L_1 + 30.0*L_10 + 30.0*L_11 + 10.0*L_12 + 10.0*L_13 + 10.0*L_14 + 10.0*L_15 + 10.0*L_16 + 10.0*L_17 + 30.0*L_18 + 30.0*L_19 + 10.0*L_2 + 30.0*L_3 + 10.0*L_4 + 30.0*L_5 + 10.0*L_6 + 30.0*L_7 + 30.0*L_8 + 30.0*L_9 + 0.0
SUBJECT TO
_C1: - T_0 + T_1 - 344 x_0_1 >= -329

_C2: - T_0 + T_2 - 367 x_0_2 >= -352

_C3: - T_0 + T_3 - 391 x_0_3 >= -383

_C4: - T_0 + T_4 - 300 x_0_4 >= -285

_C5: - T_0 + T_5 - 399 x_0_5 >= -391

_C6: - T_0 + T_6 - 316 x_0_6 >= -301

_C7: - T_0 + T_7 - 396 x_0_7 >= -388

_C8: - T_0 + T_8 - 377 x_0_8 >= -369

_C9: - T_0 + T_9 - 377 x_0_9 >= -369

_C10: - T_0 + T_10 - 362 x_0_10 >= -354

_C11: - T_0 + T_11 - 380 x_0_11 >= -372

_C12: - T_0 + T_12 - 240 x_0_12 >= 

Affichage de l'instance, du statut de résolution et de l'objectif obtenu :

In [292]:
print('Instance "{}"{}: {} planes'.format(
    instance_name.split(os.sep)[1][:-4], '', len(P)))
print('Status: ', LpStatus[problem.status])
# utiliser cela pour nouvelles versions pulp :
# print('Status: ', LpStatus[problem.getSolutionStatus()])

print('Objective: ', problem.objective.value())
print()


Instance "airland3": 20 planes
Status:  Optimal
Objective:  0.0



Récupération des valeurs des variables de décision et affichage de la solution :

In [240]:
vars = problem.variablesDict()
# dates de début
t = [round(vars['T_'+str(i)].value()) for i in P]
Q = None
#affectations aux pistes
y = []
y = []
for i in P:
    for r in R:
        if vars['y_{}_{}'.format(i, r)].roundedValue():
            y.append(r)
# Affichage

for i in P:
    print('  Plane {} appearing at {}, landing at t={} on runway {} '
          '({}, [{}, {}]).'.format(P[i], A[i], t[i], y[i], T[i],
                                   E[i], L[i]))


  Plane 0 appearing at 0, landing at t=82 on runway 2 (82, [75, 486]).
  Plane 1 appearing at 82, landing at t=197 on runway 1 (197, [157, 628]).
  Plane 2 appearing at 59, landing at t=160 on runway 1 (160, [134, 561]).
  Plane 3 appearing at 28, landing at t=117 on runway 2 (117, [103, 565]).
  Plane 4 appearing at 126, landing at t=261 on runway 1 (261, [201, 735]).
  Plane 5 appearing at 20, landing at t=106 on runway 1 (106, [95, 524]).
  Plane 6 appearing at 110, landing at t=229 on runway 2 (229, [185, 664]).
  Plane 7 appearing at 23, landing at t=108 on runway 0 (108, [98, 523]).
  Plane 8 appearing at 42, landing at t=132 on runway 1 (132, [117, 578]).
  Plane 9 appearing at 42, landing at t=130 on runway 2 (130, [117, 569]).
  Plane 10 appearing at 57, landing at t=149 on runway 0 (149, [132, 615]).
  Plane 11 appearing at 39, landing at t=126 on runway 0 (126, [114, 551]).
  Plane 12 appearing at 186, landing at t=336 on runway 0 (336, [261, 834]).
  Plane 13 appearing at 1

Il est possible de vérifier la faisabilité de la solution en appelant la fonction <tt>verifier.verify</tt>

In [241]:
p, w = verifier.verify(t, y, E, T, L, S, G, H)
print('Ok (obj. = {}).'.format(w) if p else w)

i=0 diff=0
i=1 diff=0
i=2 diff=0
i=3 diff=0
i=4 diff=0
i=5 diff=0
i=6 diff=0
i=7 diff=0
i=8 diff=0
i=9 diff=0
i=10 diff=0
i=11 diff=0
i=12 diff=0
i=13 diff=0
i=14 diff=0
i=15 diff=0
i=16 diff=0
i=17 diff=0
i=18 diff=0
i=19 diff=0
Ok (obj. = 0.0).


## 2. Le problème stochastique


Les dates d'arrivée au plus tôt, au plus tard et préférentielles $e_i$, $d_i$ et $t_i$ des avions sont maintenant incertaines et remplacées par des variables aléatoires $\tilde e_i$, $\tilde t_i$, $\tilde d_i$ suivant des scénarios discrets. On a donc un ensemble de scénarios $S$ tel qu'un scénario $s\in S$ fournit les dates  $(r_{is})_{i\in P}$, $(d_{is})_{i\in P}$ et $(t_{is})_{i\in P}$ avec une probabilité $\pi_s\geq 0$ avec $\sum_{s\in S}\pi_{s}=100\%$.
L'ensemble des scénarios fournis en séance est tel que le premier scénario correspond aux données initiales des instances <tt>airland1</tt>,..., <tt>airand13</tt>.
On considère ici que les contraintes de planification imposent de définir l'affectation des avions aux pistes et l'ordre d'atterrissage des avions avant de connaitre précisément leurs dates d'arrivée. Les dates d'atterrissage effectives seront décalées en fonction du scénario réalisé.

<i>Exemple : Dans le cours 1, on donne les dates d'atterrissages obtenues par une séquence sur deux scénarios (ou seules les dates d'arrivée au plus tôt sont incertaines) pour le problème <tt>airland1</tt>, de coût moyen 1444.</i>

Dans la cellule ci-dessous on relit un fichier d'instance et on choisit un fichier de scénarios.
Il y a 3 fichiers de scénarios différents pour chacune instances <tt>airland1</tt>, <tt>airland2</tt> et <tt>airland3</tt>. Chaque fichier de scénarios contient un jeu de trois scénarios. A ces trois scénarios s'ajoutent celui correspondant aux données initiales de l'instance associée. Chaque fichier de scénarios permet donc de considérer un ensemble de quatre scénarios. Vous pouvez  choisir un des trois fichiers de scénarios via leur numéro respectif 1, 2 ou 3 à travers la variable <tt>scenario_id</tt>. L'exécution de la cellule suivante permet d'afficher les caractéristiques des quatres scénarios associés à ce fichier avec les dates d'atterrissage au plus tôt, au plus tard et préférentielles.

In [280]:
# lecture d'un fichier d'instance et initialisation du nombre de pistes
instance_id = 'airland3'
instance_name='instances/'+instance_id+'.txt'
instance = open(instance_name)

runways = 3

P, F, A, E, L, T, S, G, H = read_instance(instance)
P = range(P)
R = range(runways)

# lecture d'un fichier de scenario venant compléter le scenario nominal donne par le fichier d'instance
scenario_id='3'
scenario_set_name = 'scenarios/'+instance_id+'_'+scenario_id+'.txt'
scenario_set = open(scenario_set_name)

bE, bT, bL = E, T, L  # Base value
P2, p, E, T, L = read_scenarios(scenario_set)
if P2 != len(P):
    print('Error: Number of planes in scenarios and instance files '
            'do not match.')
    exit(1)

p.append(1 - sum(p)) # calcul de la probabilite du scenario nominal (celui du fichier de l'instance)
for i in P:
    E[i].append(bE[i])
    T[i].append(bT[i])
    L[i].append(bL[i])

# ajout des informations sur les scénarios et les probabilités dans les arguments de la finction de définition du problème
kargs = {'P': P, 'R': R, 'e': E, 'd': L, 't': T, 's': S, 'g': G, 'h': H}
kargs['S'] = range(len(p))
kargs['p'] = p

# affichage des scenarios:
print ('nb scenarios='+str(len(p)))
print()
print('probabilités p='+str(p))
print()
print('Scénarios des dates d\'atterrissage au plus tôt:')
for k in range(len(p)):
    print('E[.]['+str(k)+']=',end="")
    for i in P:
        print(E[i][k], end =" ")
    print()
print()
print('Scénarios des dates d\'atterrissage au plus tard:')
for k in range(len(p)):
    print('L[.]['+str(k)+']=',end="")
    for i in P:
        print(L[i][k], end =" ")
    print()
print()
print('Scénarios des dates d\'atterrissage préférentielles:')
for k in range(len(p)):
    print('T[.]['+str(k)+']=',end="")
    for i in P:
        print(T[i][k], end =" ")


nb scenarios=4

probabilités p=[0.1, 0.5, 0.2, 0.19999999999999996]

Scénarios des dates d'atterrissage au plus tôt:
E[.][0]=79 150 146 97 204 103 176 103 126 120 127 125 260 273 214 336 293 232 151 158 
E[.][1]=72 154 122 107 218 103 201 106 121 123 139 104 250 242 232 325 284 218 137 154 
E[.][2]=76 171 139 95 218 100 190 92 118 126 123 117 251 251 224 313 286 220 152 161 
E[.][3]=75 157 134 103 201 95 185 98 117 117 132 114 261 250 214 310 269 237 144 151 

Scénarios des dates d'atterrissage au plus tard:
L[.][0]=354 732 701 430 867 471 756 500 628 600 568 622 1139 1333 882 1371 1296 966 740 729 
L[.][1]=345 646 507 457 1077 436 943 490 544 588 647 490 1030 988 1010 1440 1332 1067 628 766 
L[.][2]=363 696 657 474 1008 495 950 445 582 527 582 523 1240 1037 923 1315 1305 913 715 764 
L[.][3]=486 628 561 565 735 524 664 523 578 569 615 551 834 790 688 967 818 726 607 624 

Scénarios des dates d'atterrissage préférentielles:
T[.][0]=94 168 173 115 241 117 205 116 147 144 153 138 305 323

### Question 2.1

<i> Transformer le PLNE obtenue à la question 1.3 en un **programme stochastique à deux niveaux** pour minimiser l'espérance du coût. Justifier votre modélisation.</i>

 
        


### Question 2.2

<i>Programmer votre nouveau PLNE dans la fonction <tt>get_problem_q2</tt> et tester le sur les trois premières instances fournies en utilisant les scénarios fournis. Faire un tableau pour chacune des instances comparant les différentes exécutions sur différents scénarios, nombre de pistes d'aterrissage, temps de calcul et la comparaison avec la solution sur le scénario nominal.

In [281]:
def get_problem_q2(**kargs):

    P = kargs['P']  # Liste d'avions (0, 1, 2, ..., N)
    R = kargs['R']  # Pistes d'atterrissage (0, 1, 2, ..., M)
    S = kargs['S']  # Liste de scénarios
    p = kargs['p']  # Probabilité de chaque scénario
    e = kargs['e']  # Dates d'atterrissage au plus tôt
    d = kargs['d']  # Dates d'atterrissage au plus tard
    t = kargs['t']  # Dates d'atterrissage préférentielles
    s = kargs['s']  # Délais de séparation
    g = kargs['g']  # Coûts unitaires de retard
    h = kargs['h']  # Coûts unitaires d'avance

    # -- Calcul des bigs M à faire
    M = [[[d[i][k] - e[j][k]for k in S] for j in P]for i in P]
    

    # -- Création du problème
    problem = LpProblem("Ordonnancement des atterrissages - Le problème stochastique",
                        LpMinimize)

    # -- Création des variables, on vous donne T, trouver E, L, X, y

    # T - Date d'atterrissage
    T = [[LpVariable('T_{}_{}'.format(i, k), e[i][k], d[i][k]) for k in S] for i in P]

    # E - Avance de l'avion
    E = [[LpVariable('E_{}_{}'.format(i,k), 0) for k in S] for i in P]

    # L - Retard de l'avion
    L = [[LpVariable('L_{}_{}'.format(i,k), 0) for k in S] for i in P]

    # x
    x = [[LpVariable('x_{}_{}'.format(i, j), cat=LpBinary) for j in P] for i in P]
    
    # y  
    y = [[LpVariable('y_{}_{}'.format(i, r), cat=LpBinary) for r in R] for i in P]

    # -- Création de l'objectif à faire

    problem += lpSum((p[k] * (E[i][k]*h[i]+L[i][k]*g[i])for i in P) for k in S)



    # -- Création des contraintes à faire

    # Contraintes (4)
    for k in S:
        for i in P:
            for j in filter(lambda x: x != i, P):
                problem += T[j][k] >= T[i][k] + s[i][j] * x[i][j] - M[i][j][k] * (1 - x[i][j])
            
    # Contraintes (5) et (6)
    for k in S: 
        for i in P:
            problem += L[i][k] >= T[i][k] - t[i][k]
            problem += E[i][k] >= t[i][k] - T[i][k]
            
    
    # Contraintes de sequensement des avions !
    for i in P:
        for j in filter(lambda x: x != i, P):
            for r in R:
                problem += x[i][j] + x[j][i] >= y[i][r] + y[j][r] - 1

    # Contraintes d'affectation des avions aux pistes
    for i in P:
        problem += lpSum(y[i][r] for r in R) == 1
    
    #print(problem)

    return problem




Définition du nouveau problème et résolution.

In [282]:
problem = get_problem_q2(**kargs)
status = problem.solve(pulp.PULP_CBC_CMD(msg=False,maxSeconds=60))

Affichage du résultat de l'optimisation

In [283]:
sc = ' (scenarios "{}")'.format(scenario_set_name.split(os.sep)[1][:-4])
print('Instance "{}"{}: {} planes'.format(
    instance_name.split(os.sep)[1][:-4], sc, len(P)))
print('Status: ', LpStatus[problem.status])
print('Objective: ', problem.objective.value())


Instance "airland3" (scenarios "airland3_3"): 20 planes
Status:  Optimal
Objective:  1137.0


Récupération des valeurs des variables et affichage de la solution

In [284]:
print('Affectations: ')
vars = problem.variablesDict()
y = []
for i in P:
    for r in R:
        if vars['y_{}_{}'.format(i, r)].roundedValue():
            y.append(r)
Q = range(len(p))
t = [[round(vars['T_{}_{}'.format(i, s)].value()) for s in Q] for i in P]
for q in Q:
    print('  Scenario {}: '.format(q))
    for i in P:
        print('    Plane {} appearing at {}, landing at t={} on runway {} '
                '({}, [{}, {}]).'.format(P[i], A[i], t[i][q], y[i],
                                        T[i][q], E[i][q], L[i][q]))


Affectations: 
  Scenario 0: 
    Plane 0 appearing at 0, landing at t=94 on runway 2 (94, [79, 354]).
    Plane 1 appearing at 82, landing at t=168 on runway 2 (168, [150, 732]).
    Plane 2 appearing at 59, landing at t=173 on runway 1 (173, [146, 701]).
    Plane 3 appearing at 28, landing at t=115 on runway 2 (115, [97, 430]).
    Plane 4 appearing at 126, landing at t=241 on runway 1 (241, [204, 867]).
    Plane 5 appearing at 20, landing at t=117 on runway 0 (117, [103, 471]).
    Plane 6 appearing at 110, landing at t=205 on runway 2 (205, [176, 756]).
    Plane 7 appearing at 23, landing at t=107 on runway 2 (116, [103, 500]).
    Plane 8 appearing at 42, landing at t=147 on runway 0 (147, [126, 628]).
    Plane 9 appearing at 42, landing at t=182 on runway 0 (144, [120, 600]).
    Plane 10 appearing at 57, landing at t=153 on runway 2 (153, [127, 568]).
    Plane 11 appearing at 39, landing at t=138 on runway 1 (138, [125, 622]).
    Plane 12 appearing at 186, landing at t=305

Il est également possible de vérifier la validité de solution et d'avoir la valeur de l'objectif pour chaque scénario

In [285]:
# Verify for each scenario:
verif = True
for q in Q:
    u = ' (p = {:.2f})'.format(p[q])
    c, w = verifier.verify(t, y, E, T, L, S, G, H, q)
    print('Scenario {}{}: '.format(q, u), end='')
    print('Ok (obj. = {})'.format(w) if c else w)
    verif = verif and c


i=0 diff=0
i=1 diff=0
i=2 diff=0
i=3 diff=0
i=4 diff=0
i=5 diff=0
i=6 diff=0
i=7 diff=-9
i=8 diff=0
i=9 diff=38
i=10 diff=0
i=11 diff=0
i=12 diff=0
i=13 diff=0
i=14 diff=0
i=15 diff=0
i=16 diff=0
i=17 diff=0
i=18 diff=-8
i=19 diff=0
Scenario 0 (p = 0.10): Ok (obj. = 1650.0)
i=0 diff=0
i=1 diff=0
i=2 diff=0
i=3 diff=0
i=4 diff=0
i=5 diff=0
i=6 diff=0
i=7 diff=-5
i=8 diff=0
i=9 diff=29
i=10 diff=0
i=11 diff=0
i=12 diff=0
i=13 diff=0
i=14 diff=0
i=15 diff=0
i=16 diff=0
i=17 diff=0
i=18 diff=0
i=19 diff=0
Scenario 1 (p = 0.50): Ok (obj. = 1020.0)
i=0 diff=0
i=1 diff=0
i=2 diff=0
i=3 diff=0
i=4 diff=0
i=5 diff=0
i=6 diff=0
i=7 diff=-5
i=8 diff=0
i=9 diff=27
i=10 diff=0
i=11 diff=0
i=12 diff=0
i=13 diff=0
i=14 diff=0
i=15 diff=0
i=16 diff=0
i=17 diff=0
i=18 diff=-7
i=19 diff=0
Scenario 2 (p = 0.20): Ok (obj. = 1170.0)
i=0 diff=0
i=1 diff=0
i=2 diff=0
i=3 diff=0
i=4 diff=0
i=5 diff=0
i=6 diff=0
i=7 diff=0
i=8 diff=0
i=9 diff=31
i=10 diff=0
i=11 diff=0
i=12 diff=0
i=13 diff=0
i=14 diff=0
i=15 

## 3. Le problème robuste

On considère le cas où on dispose des mêmes scénarios de dates d'arrivée des avions que pour le problème stochastique mais où on ne peut les associer à aucune probabilité.

<i> Exemple : Dans le cours 1, on montre les dates d'atterrissages obtenue par une séquence sur les mêmes scénarios pour le problème <tt>airland1</tt>, de coût maximal 1610.</i>

### Question 3.1

<i>Transformer le PLNE obtenue à la question 1.3 pour minimiser le coût dans le pire scénario. Justifier votre modélisation.</i>


         


### Question 3.2

<i>Programmer votre nouveau PLNE dans la fonction et comparer les solutions obtenues par les PLNE statique, stochastique et robuste sur les trois premières instances et leurs scénarios.</i>

In [286]:
def get_problem_q3(**kargs):

    P = kargs['P']  # Liste d'avions (0, 1, 2, ..., N)
    R = kargs['R']  # Pistes d'atterrissage (0, 1, 2, ..., M)
    S = kargs['S']  # Liste de scénarios
    e = kargs['e']  # Dates d'atterrissage au plus tôt
    d = kargs['d']  # Dates d'atterrissage au plus tard
    t = kargs['t']  # Dates d'atterrissage préférentielles
    s = kargs['s']  # Délais de séparation
    g = kargs['g']  # Coûts unitaires de retard
    h = kargs['h']  # Coûts unitaires d'avance

    # -- Calcul des bigs M a faire
    M = [[[d[i][k] - e[j][k]for k in S] for j in P]for i in P]


    # -- Création du problème
    problem = LpProblem("Ordonnancement des atterrissages - Le problème robuste",
                        LpMinimize)

    # -- Création des variables on vous donne la variable T ... créer les autres (E, L, x, y)

    # T - Date d'atterrissage
    T = [[LpVariable('T_{}_{}'.format(i, k), e[i][k], d[i][k]) for k in S] for i in P]

    # E - Avance de l'avion
    E = [[LpVariable('E_{}_{}'.format(i, k), 0) for k in S] for i in P]
    
    # L - Retard de l'avion
    L = [[LpVariable('L_{}_{}'.format(i, k), 0) for k in S] for i in P]

    # x
    x = [[LpVariable('x_{}_{}'.format(i, j), cat=LpBinary) for j in P] for i in P]
    
    # y  
    y = [[LpVariable('y_{}_{}'.format(i, r), cat=LpBinary) for r in R] for i in P]
    
    z = LpVariable('z')    
     

    # -- Création de l'objectif à faire
    
    problem += z
    
    # -- Création des contraintes à faire
    for k in S:
        problem += z >= lpSum([g[i]*L[i][k] + h[i]*E[i][k] for i in P])
    
    for k in S:
        for i in P:
            for j in filter(lambda x: x != i, P):
                problem += T[j][k] >= T[i][k] + s[i][j] * x[i][j] - M[i][j][k] * (1 - x[i][j])
    
    
    for i in P:
        for j in filter(lambda x: x != i, P):
            for r in R:
                problem += x[i][j] + x[j][i] >= y[i][r] + y[j][r] - 1
                
    
    
    for k in S:
        for i in P:
            problem += L[i][k] >= T[i][k] - t[i][k]
            problem += E[i][k] >= t[i][k] - T[i][k]
            
    for i in P:
        problem += lpSum([y[i][r] for r in R]) == 1 
    

    print(problem)
    return problem


Définition du nouveau problème et résolution.

In [287]:
problem = get_problem_q3(**kargs)
status = problem.solve(pulp.PULP_CBC_CMD(msg=True,maxSeconds=60))

Ordonnancement_des_atterrissages_-_Le_problème_robuste:
MINIMIZE
1*z + 0
SUBJECT TO
_C1: - 30 E_0_0 - 30 E_10_0 - 30 E_11_0 - 10 E_12_0 - 10 E_13_0 - 10 E_14_0
 - 10 E_15_0 - 10 E_16_0 - 10 E_17_0 - 30 E_18_0 - 30 E_19_0 - 10 E_1_0
 - 10 E_2_0 - 30 E_3_0 - 10 E_4_0 - 30 E_5_0 - 10 E_6_0 - 30 E_7_0 - 30 E_8_0
 - 30 E_9_0 - 30 L_0_0 - 30 L_10_0 - 30 L_11_0 - 10 L_12_0 - 10 L_13_0
 - 10 L_14_0 - 10 L_15_0 - 10 L_16_0 - 10 L_17_0 - 30 L_18_0 - 30 L_19_0
 - 10 L_1_0 - 10 L_2_0 - 30 L_3_0 - 10 L_4_0 - 30 L_5_0 - 10 L_6_0 - 30 L_7_0
 - 30 L_8_0 - 30 L_9_0 + z >= 0

_C2: - 30 E_0_1 - 30 E_10_1 - 30 E_11_1 - 10 E_12_1 - 10 E_13_1 - 10 E_14_1
 - 10 E_15_1 - 10 E_16_1 - 10 E_17_1 - 30 E_18_1 - 30 E_19_1 - 10 E_1_1
 - 10 E_2_1 - 30 E_3_1 - 10 E_4_1 - 30 E_5_1 - 10 E_6_1 - 30 E_7_1 - 30 E_8_1
 - 30 E_9_1 - 30 L_0_1 - 30 L_10_1 - 30 L_11_1 - 10 L_12_1 - 10 L_13_1
 - 10 L_14_1 - 10 L_15_1 - 10 L_16_1 - 10 L_17_1 - 30 L_18_1 - 30 L_19_1
 - 10 L_1_1 - 10 L_2_1 - 30 L_3_1 - 10 L_4_1 - 30 L_5_1 - 10 L_6_

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mtds/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/4112ba3a4d7d4384bb5558c25e156585-pulp.mps sec 60 timeMode elapsed branch printingOptions all solution /tmp/4112ba3a4d7d4384bb5558c25e156585-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2849 COLUMNS
At line 13395 RHS
At line 16240 BOUNDS
At line 16842 ENDATA
Problem MODEL has 2844 rows, 681 columns and 9664 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 75 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 38 strengthened rows, 0 substitutions
Cgl0004I processed model has 2274 rows, 681 columns (440 integer (440 of which binary)) and 7497 elements
Cbc0038I Initial state - 40 integers unsat

Cbc0010I After 5700 nodes, 144 on tree, 330 best solution, best possible 0 (50.39 seconds)
Cbc0010I After 5800 nodes, 143 on tree, 330 best solution, best possible 0 (50.90 seconds)
Cbc0010I After 5900 nodes, 144 on tree, 330 best solution, best possible 0 (51.54 seconds)
Cbc0038I Full problem 2274 rows 681 columns, reduced to 902 rows 374 columns
Cbc0010I After 6000 nodes, 131 on tree, 330 best solution, best possible 0 (52.26 seconds)
Cbc0010I After 6100 nodes, 126 on tree, 330 best solution, best possible 0 (53.33 seconds)
Cbc0010I After 6200 nodes, 117 on tree, 330 best solution, best possible 0 (54.39 seconds)
Cbc0010I After 6300 nodes, 118 on tree, 330 best solution, best possible 0 (55.29 seconds)
Cbc0010I After 6400 nodes, 107 on tree, 330 best solution, best possible 0 (55.96 seconds)
Cbc0010I After 6500 nodes, 106 on tree, 330 best solution, best possible 0 (56.88 seconds)
Cbc0010I After 6600 nodes, 109 on tree, 330 best solution, best possible 0 (57.58 seconds)
Cbc0010I Afte

Affichage du résultat de l'optimisation

In [288]:
sc = ' (scenarios "{}")'.format(scenario_set_name.split(os.sep)[1][:-4])
print('Instance "{}"{}: {} planes'.format(
    instance_name.split(os.sep)[1][:-4], sc, len(P)))
print('Status: ', LpStatus[problem.status])
print('Objective: ', problem.objective.value())


Instance "airland3" (scenarios "airland3_3"): 20 planes
Status:  Optimal
Objective:  310.0


Récupération des valeurs des variables et affichage de la solution

In [20]:
print('Affectations: ')
vars = problem.variablesDict()
print(vars)
y = []
for i in P:
    for r in R:
        if vars['y_{}_{}'.format(i, r)].roundedValue():
            y.append(r)
Q = range(len(p))
t = [[round(vars['T_{}_{}'.format(i, s)].value()) for s in Q] for i in P]
for q in Q:
    print('  Scenario {}: '.format(q))
    for i in P:
        print('    Plane {} appearing at {}, landing at t={} on runway {} '
                '({}, [{}, {}]).'.format(P[i], A[i], t[i][q], y[i],
                                        T[i][q], E[i][q], L[i][q]))


Affectations: 
{'z': z, 'L_0_0': L_0_0, 'E_0_0': E_0_0, 'L_1_0': L_1_0, 'E_1_0': E_1_0, 'L_2_0': L_2_0, 'E_2_0': E_2_0, 'L_3_0': L_3_0, 'E_3_0': E_3_0, 'L_4_0': L_4_0, 'E_4_0': E_4_0, 'L_5_0': L_5_0, 'E_5_0': E_5_0, 'L_6_0': L_6_0, 'E_6_0': E_6_0, 'L_7_0': L_7_0, 'E_7_0': E_7_0, 'L_8_0': L_8_0, 'E_8_0': E_8_0, 'L_9_0': L_9_0, 'E_9_0': E_9_0, 'L_10_0': L_10_0, 'E_10_0': E_10_0, 'L_11_0': L_11_0, 'E_11_0': E_11_0, 'L_12_0': L_12_0, 'E_12_0': E_12_0, 'L_13_0': L_13_0, 'E_13_0': E_13_0, 'L_14_0': L_14_0, 'E_14_0': E_14_0, 'L_15_0': L_15_0, 'E_15_0': E_15_0, 'L_16_0': L_16_0, 'E_16_0': E_16_0, 'L_17_0': L_17_0, 'E_17_0': E_17_0, 'L_18_0': L_18_0, 'E_18_0': E_18_0, 'L_19_0': L_19_0, 'E_19_0': E_19_0, 'L_0_1': L_0_1, 'E_0_1': E_0_1, 'L_1_1': L_1_1, 'E_1_1': E_1_1, 'L_2_1': L_2_1, 'E_2_1': E_2_1, 'L_3_1': L_3_1, 'E_3_1': E_3_1, 'L_4_1': L_4_1, 'E_4_1': E_4_1, 'L_5_1': L_5_1, 'E_5_1': E_5_1, 'L_6_1': L_6_1, 'E_6_1': E_6_1, 'L_7_1': L_7_1, 'E_7_1': E_7_1, 'L_8_1': L_8_1, 'E_8_1': E_8_1, 'L_9_1':

Vérification de la validité de solution et affichage de la valeur de l'objectif pour chaque scénario

In [21]:
# Verify for each scenario:
verif = True
for q in Q:
    u = ''
    c, w = verifier.verify(t, y, E, T, L, S, G, H, q)
    print('Scenario {}{}: '.format(q, u), end='')
    print('Ok (obj. = {})'.format(w) if c else w)
    verif = verif and c


i=0 diff=0
i=1 diff=17
i=2 diff=12
i=3 diff=0
i=4 diff=0
i=5 diff=0
i=6 diff=-12
i=7 diff=0
i=8 diff=0
i=9 diff=4
i=10 diff=0
i=11 diff=0
i=12 diff=0
i=13 diff=0
i=14 diff=0
i=15 diff=0
i=16 diff=0
i=17 diff=-13
i=18 diff=0
i=19 diff=0
Scenario 0: Ok (obj. = 660.0)
i=0 diff=0
i=1 diff=7
i=2 diff=3
i=3 diff=13
i=4 diff=0
i=5 diff=-4
i=6 diff=0
i=7 diff=0
i=8 diff=0
i=9 diff=-4
i=10 diff=0
i=11 diff=-10
i=12 diff=9
i=13 diff=0
i=14 diff=0
i=15 diff=0
i=16 diff=0
i=17 diff=-2
i=18 diff=0
i=19 diff=0
Scenario 1: Ok (obj. = 1140.0)
i=0 diff=0
i=1 diff=3
i=2 diff=0
i=3 diff=-3
i=4 diff=0
i=5 diff=0
i=6 diff=0
i=7 diff=0
i=8 diff=0
i=9 diff=-10
i=10 diff=0
i=11 diff=0
i=12 diff=72
i=13 diff=0
i=14 diff=0
i=15 diff=0
i=16 diff=0
i=17 diff=0
i=18 diff=0
i=19 diff=0
Scenario 2: Ok (obj. = 1140.0)
i=0 diff=0
i=1 diff=0
i=2 diff=-6
i=3 diff=0
i=4 diff=0
i=5 diff=0
i=6 diff=0
i=7 diff=0
i=8 diff=-7
i=9 diff=3
i=10 diff=0
i=11 diff=0
i=12 diff=0
i=13 diff=0
i=14 diff=0
i=15 diff=0
i=16 diff=0
i=17 d